In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sust-dl-enigma-1-0-3-fold-cv-dataset/BadODD/badodd.txt
/kaggle/input/sust-dl-enigma-1-0-3-fold-cv-dataset/BadODD/labels/train/fold1/khulna7_22170.txt
/kaggle/input/sust-dl-enigma-1-0-3-fold-cv-dataset/BadODD/labels/train/fold1/chuadanga_sirajganj1_28740.txt
/kaggle/input/sust-dl-enigma-1-0-3-fold-cv-dataset/BadODD/labels/train/fold1/dhaka_night3_29913.txt
/kaggle/input/sust-dl-enigma-1-0-3-fold-cv-dataset/BadODD/labels/train/fold1/sylhet4_40887.txt
/kaggle/input/sust-dl-enigma-1-0-3-fold-cv-dataset/BadODD/labels/train/fold1/khulna4_21420.txt
/kaggle/input/sust-dl-enigma-1-0-3-fold-cv-dataset/BadODD/labels/train/fold1/maowa_expressway2_26727.txt
/kaggle/input/sust-dl-enigma-1-0-3-fold-cv-dataset/BadODD/labels/train/fold1/maowa_expressway2_47318.txt
/kaggle/input/sust-dl-enigma-1-0-3-fold-cv-dataset/BadODD/labels/train/fold1/sylhet4_2537.txt
/kaggle/input/sust-dl-enigma-1-0-3-fold-cv-dataset/BadODD/labels/train/fold1/khulna2_928.txt
/kaggle/input/sust-dl-enigma-1-0-3-fold-c

In [9]:
import pandas as pd
import torch as torch
import numpy as np
from glob import glob
from tqdm import tqdm
import cv2, os, shutil

In [10]:
!pip install ultralytics -q

In [11]:
from ultralytics import YOLO
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

Setup complete. Using torch 2.0.0 _CudaDeviceProperties(name='Tesla P100-PCIE-16GB', major=6, minor=0, total_memory=16276MB, multi_processor_count=56)


In [21]:
!rm -rf *

# make directory ...

!mkdir /kaggle/working/datasets
!mkdir /kaggle/working/datasets/badodd
!mkdir /kaggle/working/datasets/badodd/labels
!mkdir /kaggle/working/datasets/badodd/labels/train
!mkdir /kaggle/working/datasets/badodd/images
!mkdir /kaggle/working/datasets/badodd/images/train
!mkdir /kaggle/working/datasets/badodd/images/test

In [22]:
all_files_in_folder_symlink("/kaggle/input/dl-enigma-10-sust-cse-carnival-2024/dlenigma1/BadODD/labels/train","/kaggle/working/datasets/badodd/labels/train")
all_files_in_folder_symlink("/kaggle/input/dl-enigma-10-sust-cse-carnival-2024/dlenigma1/BadODD/images/train","/kaggle/working/datasets/badodd/images/train")
all_files_in_folder_symlink("/kaggle/input/dl-enigma-10-sust-cse-carnival-2024/dlenigma1/BadODD/images/test","/kaggle/working/datasets/badodd/images/test")


100%|██████████| 1964/1964 [00:00<00:00, 28041.03it/s]


In [23]:
# working directory structure ...

!tree -d

.
`-- datasets
    `-- badodd
        |-- images
        |   |-- test
        |   `-- train
        `-- labels
            `-- train

7 directories


In [24]:
file_content = """
path: /kaggle/working/datasets/badodd  # dataset root dir
train: images/train  # train images (relative to 'path')
val: images/train  # val images (relative to 'path')
test:  images/test

# Classes
names:
    0: auto_rickshaw
    1: bicycle
    2: bus
    3: car
    4: cart_vehicle
    5: construction_vehicle
    6: motorbike
    7: person
    8: priority_vehicle
    9: three_wheeler
    10: train
    11: truck
    12: wheelchair
"""

with open("yolov8.yaml", mode="w") as f:
    f.write(file_content)

In [26]:
import wandb
wandb.init(mode="disabled")

In [27]:
def all_files_in_folder_symlink(source_dir, target_dir):
    files = os.listdir(source_dir)
    
    for file in tqdm(files):
        source_file = os.path.join(source_dir, file)
        target_file = os.path.join(target_dir, file)
        os.symlink(source_file, target_file)

In [ ]:
model = YOLO("yolov8l.pt")
model.train(data='/kaggle/working/yolov8.yaml', epochs=50,
    imgsz=640,               # Size of input images as integer
    device=0,                # Device to run on, i.e. cuda device=0 
    patience=50,             # Epochs to wait for no observable improvement for early stopping of training
    batch=16,                # Number of images per batch
    optimizer='auto',        # Optimizer to use, choices=[SGD, Adam, Adamax, AdamW, NAdam, RAdam, RMSProp, auto]
    lr0=0.0001,              # Initial learning rate 
    lrf=0.1,                 # Final learning rate (lr0 * lrf)
    dropout=0.1,             # Use dropout regularization
    seed=0 
)
import os
os.system("nvidia-smi")

100%|██████████| 83.7M/83.7M [00:00<00:00, 299MB/s]


Ultralytics YOLOv8.1.6 🚀 Python-3.10.12 torch-2.0.0 CUDA:0 (Tesla P100-PCIE-16GB, 16276MiB)
engine/trainer: task=detect, mode=train, model=yolov8l.pt, data=/kaggle/working/yolov8.yaml, epochs=50, time=None, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.1, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, sho

100%|██████████| 755k/755k [00:00<00:00, 19.2MB/s]
2024-01-28 12:20:14,519	INFO util.py:129 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-01-28 12:20:15,011	INFO util.py:129 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Overriding model.yaml nc=80 with nc=13

                   from  n    params  module                                       arguments                     
  0                  -1  1      1856  ultralytics.nn.modules.conv.Conv             [3, 64, 3, 2]                 
  1                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  2                  -1  3    279808  ultralytics.nn.modules.block.C2f             [128, 128, 3, True]           
  3                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  4                  -1  6   2101248  ultralytics.nn.modules.block.C2f             [256, 256, 6, True]           
  5                  -1  1   1180672  ultralytics.nn.modules.conv.Conv             [256, 512, 3, 2]              
  6                  -1  6   8396800  ultralytics.nn.modules.block.C2f             [512, 512, 6, True]           
  7                  -1  1   2360320  ultralytic

100%|██████████| 6.23M/6.23M [00:00<00:00, 169MB/s]


...

train: Scanning /kaggle/working/datasets/badodd/labels/train... 5896 images, 0 backgrounds, 0 corrupt: 100%|██████████| 5896/5896 [00:27<00:00, 217.19it/s]


train: New cache created: /kaggle/working/datasets/badodd/labels/train.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


val: Scanning /kaggle/working/datasets/badodd/labels/train.cache... 5896 images, 0 backgrounds, 0 corrupt: 100%|██████████| 5896/5896 [00:00<?, ?it/s]


Plotting labels to runs/detect/train/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.0001' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000588, momentum=0.9) with parameter groups 97 weight(decay=0.0), 104 weight(decay=0.0005), 103 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 4 dataloader workers
Logging results to runs/detect/train
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/50      10.5G      1.239      1.492      1.157        174        640:  49%|████▉     | 182/369 [02:52<02:52,  1.08it/s]